# Analysis for PoST Parameters
The proof consists of a single nonce and several "good" labels.

In [106]:
from IPython.display import display,Math,Markdown
from scipy.stats import binom
import math
from math import log, floor, ceil, sqrt

def dm(str):
    display(Markdown(str))


## Honest Parameters

In [196]:
n = 2**34 # Total number of labels

honest_nonces = 6 # Maximum number of honest nonces

honest_opt_error = 2**(-10) # Honest users should fail with at most this probability with {honest_nonces} nonces.
honest_min_error = 2**(-20) # We also compute the number of nonces to achieve this error.

## Adversarial Parameters

We use these parameters to compute the minimal number of good labels needed.

In [197]:
# maximum number of adversarial nonces. Trying this number of nonces should be as expensive as initializing all of the adversary's claimed space.
adv_nonces = 2**40

# Fraction of storage that the adversary keeps
c = 0.75 

In [198]:

# Display the calculations for a proof that requires $k$ good labels out of $n$, where each label is good with probability $p$.
def display_results(k, n, p):
    actualq = binom.sf(k,n,p) # Actual probability of finding at least k good labels out of n (might differ from q due to rounding)
    adv_prob = binom.sf(k, ceil(c*n), p)
    adv_log_prob = binom.logsf(k, ceil(c*n), p)/log(2)

    if actualq == 0 or actualq == 1 or log(1-actualq) == 0:
        dm(rf"**Error**: actualq=${actualq:.2f}=2^{{ {log(actualq,2):.2f} }}$")
        return
    
    opt_nonces = ceil(log(honest_opt_error)/log(1-actualq))
    honest_actual_error = (1-actualq)**opt_nonces

    max_nonces = ceil(log(honest_min_error)/log(1-actualq))

    adv_success_log_prob = binom.logsf(0, adv_nonces, adv_prob)/log(2) # We use the binom.logsf function rather than directly computing because it handles the underflows more nicely.

    
    dm(rf"### Proof consists of a single nonce with {k} good labels")
    dm(rf"For honest users: $q=\Pr[Binom(n=2^{{ {log(n,2):.2f} }}, p={p*n:.2f}/n) \ge {k}] = {binom.sf(k,n,p):.2f}$")
    dm(rf"  *  Need {opt_nonces} nonces to ensure honest failure probability of less than $2^{{ {log(honest_actual_error,2):.2f} }} \le 2^{{ {log(honest_opt_error,2):.2f} }}$")
    dm(rf"  *  Need {max_nonces} nonces to ensure honest failure probability of less than $2^{{ {log(honest_min_error,2):.2f} }}$")
    dm(rf"For the adversary: $\Pr[Binom({c:.2f}\cdot n, p={p*n:.2f}/n) \ge {k}] = {adv_prob:.2f} = 2^{{ {adv_log_prob:.2f} }}$")
    dm(rf"  *  Expected number of nonces for adversary is ${floor(1/adv_prob)}=2^{{ {-adv_log_prob:.2f} }}$")
    dm(rf"  *  With 2^{{ {log(adv_nonces,2):.2f} }} nonce attempts, probability that adversary succeeds is less than $2^{{ {adv_success_log_prob:.2f} }}$")


## Inverse calculations and optimizations

### Finding $p$ given $k$ and $q$

In [199]:
from scipy import optimize

k = 600
q = 0.59

# Find the p that minimizes, for a binomially distributed variable Binom with parameters n,p, the error
# |Pr[Binom(n,p) >= k] - q|. 
def find_p(k1: int, n1: int, q1: float):
    """Find p such that Pr[X \ge k]=q when X~Binom(n,p)"""
    def p_error(p):
        if p <= 0 or p >= 1:
            return math.inf
        actualq = binom.sf(k1,n1,p)
        if actualq == 0:
            return 3-p
        elif actualq == 1:
            return 2+p
        return (q1 - actualq)**2

    if q1 == 0.5:
        return k1/n1
    elif q1 < 0.5:
        bounds=(0,k1/n1)
    else:
        bounds=(k1/n1, 2*k1/n1)
        
    res = optimize.minimize_scalar(p_error,bounds=bounds, method='bounded', options={'xatol': 2**(-32), 'maxiter': 2**20})
        
    return res.x

p = find_p(k,n,q)


dm(rf"Best solution found is $p=2^{{ {log(p,2):.3f} }} = \frac{{ {p*n:.2f} }} {{ n }}$")
dm(rf"$\Pr[Binom(n=2^{{ {log(n,2):.2f} }}, p={p*n:.2f}/n \ge {k}] = {binom.sf(k,n,p):.2f}$")

Best solution found is $p=2^{ -24.755 } = \frac{ 606.56 } { n }$

$\Pr[Binom(n=2^{ 34.00 }, p=606.56/n \ge 600] = 0.59$

### Optimizing $q$ given $k$

We're trying to maximize the gap between the honest success probability and the adversarial success.

In [200]:
def find_q(k1: int, n1: int, c1: float):
    def q_error(q):
        if q <= 0 or q >= 1:
            return math.inf
        p = find_p(k1, n1, q)
        
        opt_nonces = log(honest_opt_error)/log(1-q)
        if opt_nonces > honest_nonces:
            return math.inf

        adv_logprob = binom.logsf(k1, ceil(c1*n1), p)
        honest_logprob = binom.logsf(k1, n1, p)
        
        if adv_logprob >= honest_logprob:
            return math.inf
        
        return adv_logprob-honest_logprob
    
    res = optimize.minimize_scalar(q_error,bounds=(0,1), method='bounded', options={'xatol': 2**(-32), 'maxiter': 2**20})
    return res.x



k = 600
q = find_q(k,n,c)
p = find_p(k,n,q)


dm(rf"Best solution found is $q\approx {q:.2f}=2^{{ {log(q,2):.3f} }}, p=2^{{ {log(p,2):.3f} }} = \frac{{ {p*n:.2f} }} {{ n }}$")

Best solution found is $q\approx 0.69=2^{ -0.540 }, p=2^{ -24.741 } = \frac{ 612.77 } { n }$

### Optimizing $k$

In [201]:
# Find the smallest k that requires the adversary to search through min_adv_nonces in expectation to find a solution.

def find_k(n1: int, c1: float, min_log_adv_nonces: int):
    def k_error(k):
        k = ceil(k)
        q = find_q(k, n1, c1)
        p = find_p(k, n1, q)
        
        adv_log_prob = binom.logsf(k, ceil(c*n1), p)/log(2)
        expected_log_nonces = -adv_log_prob
        
        if expected_log_nonces < min_log_adv_nonces:
            return math.inf
        else:
            return k
    
    res = optimize.minimize_scalar(k_error, bounds=(16, 1000), method='bounded', options={'xatol': 0.5, 'maxiter': 2000})
    return res.x


k = find_k(n,c,log(adv_nonces,2))
k = ceil(k)
q = find_q(k,n,c)
p = find_p(k,n,q)

display_results(k,n,p)

### Proof consists of a single nonce with 747 good labels

For honest users: $q=\Pr[Binom(n=2^{ 34.00 }, p=761.31/n) \ge 747] = 0.69$

  *  Need 6 nonces to ensure honest failure probability of less than $2^{ -10.14 } \le 2^{ -10.00 }$

  *  Need 12 nonces to ensure honest failure probability of less than $2^{ -20.00 }$

For the adverary: $\Pr[Binom(0.75\cdot n, p=761.31/n) \ge 747] = 0.00 = 2^{ -40.08 }$

  *  Expected number of nonces for adversary is $1160887398864=2^{ 40.08 }$

  *  With 2^{ 40.00 } nonce attempts, probability that adversary succeeds is less than $2^{ -0.71 }$

## Other values of $k$

In [202]:
for k in [300, 400, 500, 600, 700]:
    q = find_q(k,n,c)
    p = find_p(k,n,q)
    
    display_results(k,n,p)

### Proof consists of a single nonce with 300 good labels

For honest users: $q=\Pr[Binom(n=2^{ 34.00 }, p=309.00/n) \ge 300] = 0.68$

  *  Need 7 nonces to ensure honest failure probability of less than $2^{ -11.60 } \le 2^{ -10.00 }$

  *  Need 13 nonces to ensure honest failure probability of less than $2^{ -20.00 }$

For the adverary: $\Pr[Binom(0.75\cdot n, p=309.00/n) \ge 300] = 0.00 = 2^{ -17.01 }$

...

  *  Need 6 nonces to ensure honest failure probability of less than $2^{ -10.52 } \le 2^{ -10.00 }$

  *  Need 12 nonces to ensure honest failure probability of less than $2^{ -20.00 }$

For the adverary: $\Pr[Binom(0.75\cdot n, p=714.90/n) \ge 700] = 0.00 = 2^{ -37.28 }$

  *  Expected number of nonces for adversary is $167206463197=2^{ 37.28 }$

  *  With 2^{ 40.00 } nonce attempts, probability that adversary succeeds is less than $2^{ -0.00 }$